In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import TimeDistributed
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
!pip install netron
import netron
from tensorflow.keras.models import load_model
from random import randint

In [2]:
# Cargar modelo entrenado
model = load_model("model.h5", compile=False)

In [3]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        if out_end_ix > len(sequences):
            break
        seq_x = sequences[i:end_ix, :-1]
        seq_y = sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [4]:
data = pd.read_csv("AS2(t-12).csv")

index = data['clave']
data = data.drop(['clave'], axis=1)


In [5]:
encoder = LabelEncoder()
data[data.columns[7]] = encoder.fit_transform(data[data.columns[7]])


In [6]:
a1 = list(range(13,24))
a2 = list(range(0,24)) * 1306
a3 = list(range(0,24))
a4 = a1 + a2 + a3

data['hour'] = a4


In [7]:
data = data.reindex(columns=[
    data.columns[0], data.columns[1], data.columns[2],
    data.columns[3], data.columns[4], data.columns[5],
    data.columns[6], data.columns[7], 'hour', 'obj'
])


In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
data[data.columns.difference(['obj'])] = scaler.fit_transform(
    data[data.columns.difference(['obj'])]
)


In [9]:
data1 = data.values

X, y = split_sequences(data1, 4, 1)

X = X.reshape((X.shape[0], X.shape[1], X.shape[2])).astype('float32')


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)


In [11]:
model.input_shape

(None, 4, 9, 1)

In [12]:
import numpy as np

X_train_flat = X_train.reshape(X_train.shape[0], -1)  # (N, 36)
X_test_flat  = X_test.reshape(X_test.shape[0], -1)


In [13]:
def model_predict_flat(x_flat):
    x = x_flat.reshape(x_flat.shape[0], 4, 9)
    return model.predict(x)


In [14]:
variables = [
    "var1_AS2_flow",
    "var2_aux_flow_1",
    "var3_aux_flow_2",
    "var4_weather_1",
    "var5_weather_2",
    "var6_weather_3",
    "var7_weather_4",
    "var8_day_type",
    "hour"
]


timesteps = ["t-3", "t-2", "t-1", "t"]

feature_names = [
    f"{v}_{t}" for t in timesteps for v in variables
]


In [15]:
def integrated_gradients(model, x, baseline=None, steps=50):
    """
    x: tensor shape (1, timesteps, features)
    """
    if baseline is None:
        baseline = tf.zeros_like(x)

    x = tf.cast(x, tf.float32)
    baseline = tf.cast(baseline, tf.float32)

    # Interpolación lineal
    interpolated = [
        baseline + (i / steps) * (x - baseline)
        for i in range(steps + 1)
    ]
    interpolated = tf.concat(interpolated, axis=0)

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        preds = model(interpolated)

    grads = tape.gradient(preds, interpolated)

    avg_grads = tf.reduce_mean(grads[:-1], axis=0)
    ig = (x - baseline) * avg_grads

    return ig.numpy()


In [16]:
n_samples = 100

ig_values = []

for i in range(n_samples):
    x = X_test[i:i+1]              # shape (1,4,9)
    ig = integrated_gradients(model, x)
    ig_values.append(np.abs(ig))   # valor absoluto


In [17]:
ig_values = np.array(ig_values)    # shape (100,4,9)

In [18]:
mean_ig = ig_values.mean(axis=0)   # shape (4,9)

In [19]:
var_importance = mean_ig.mean(axis=0)   # (9,)
print(var_importance)

[[2.7095302e-04 6.7202561e-04 2.0324811e-04 6.2662852e-04 7.7263721e-05
  2.4686290e-03 1.3084575e-03 2.0339764e-03 1.0494426e-03]
 [1.9347882e-04 4.4526448e-04 1.8233308e-04 7.7086588e-04 7.9684491e-05
  3.4176381e-03 1.6698022e-03 1.3359960e-03 7.3366356e-04]
 [1.4722600e-04 3.0483471e-04 1.6728128e-04 1.0818313e-03 1.0017365e-04
  4.4725956e-03 2.2769526e-03 8.7846688e-04 6.4535590e-04]
 [1.7072372e-04 3.1100336e-04 2.1921039e-04 1.4095092e-03 1.3547360e-04
  5.7886136e-03 3.1680313e-03 8.6675631e-04 5.3077249e-04]]


In [20]:
time_importance = mean_ig.mean(axis=1)  # (4,)
print(time_importance)


[[1.9559539e-04 4.3328205e-04 1.9301822e-04 9.7220874e-04 9.8148870e-05
  4.0368689e-03 2.1058109e-03 1.2787989e-03 7.3980866e-04]]


In [21]:
timesteps = ["t-3", "t-2", "t-1", "t"]
variables = [
    "var1_AS2_flow",
    "var2_aux_flow_1",
    "var3_aux_flow_2",
    "var4_weather_1",
    "var5_weather_2",
    "var6_weather_3",
    "var7_weather_4",
    "var8_day_type",
    "hour"
]


In [22]:
var_importance = mean_ig.mean(axis=0)  # (9,)


In [23]:
for v, imp in zip(variables, var_importance):
    print(v, imp)


var1_AS2_flow [2.7095302e-04 6.7202561e-04 2.0324811e-04 6.2662852e-04 7.7263721e-05
 2.4686290e-03 1.3084575e-03 2.0339764e-03 1.0494426e-03]
var2_aux_flow_1 [1.9347882e-04 4.4526448e-04 1.8233308e-04 7.7086588e-04 7.9684491e-05
 3.4176381e-03 1.6698022e-03 1.3359960e-03 7.3366356e-04]
var3_aux_flow_2 [0.00014723 0.00030483 0.00016728 0.00108183 0.00010017 0.0044726
 0.00227695 0.00087847 0.00064536]
var4_weather_1 [0.00017072 0.000311   0.00021921 0.00140951 0.00013547 0.00578861
 0.00316803 0.00086676 0.00053077]


In [24]:
time_importance = mean_ig.mean(axis=1)  # (4,)


In [25]:
for t, imp in zip(timesteps, time_importance):
    print(t, imp)


t-3 [1.9559539e-04 4.3328205e-04 1.9301822e-04 9.7220874e-04 9.8148870e-05
 4.0368689e-03 2.1058109e-03 1.2787989e-03 7.3980866e-04]


In [26]:
import pandas as pd
mean_ig = mean_ig.squeeze()

df_ig = pd.DataFrame(
    mean_ig,
    index=timesteps,
    columns=variables
)
df_ig.to_csv("XAI_AS12.csv")

print(df_ig)


     var1_AS2_flow  var2_aux_flow_1  var3_aux_flow_2  var4_weather_1  \
t-3       0.000271         0.000672         0.000203        0.000627   
t-2       0.000193         0.000445         0.000182        0.000771   
t-1       0.000147         0.000305         0.000167        0.001082   
t         0.000171         0.000311         0.000219        0.001410   

     var5_weather_2  var6_weather_3  var7_weather_4  var8_day_type      hour  
t-3        0.000077        0.002469        0.001308       0.002034  0.001049  
t-2        0.000080        0.003418        0.001670       0.001336  0.000734  
t-1        0.000100        0.004473        0.002277       0.000878  0.000645  
t          0.000135        0.005789        0.003168       0.000867  0.000531  
